## Introduction/Business Problem:

I am a young professional who will be moving to Chicago and looking for a place to live.  My fundamental business question is where will I be happiest in Chicago?  There are a number of questions I need to concider when choosing where I will ulimately buy my new home.  

First is what is available within my budget?  Can I afford a house or a small condo? What fees other fees bring up my monthly payments?

Second what are the neigborhood characteristics available near the homes I can afford? Are there problems with crime in the area?  How close are the nearest public transpertation spots?  What social activities are in the area? e.g. bars, restaurants?   I enjoy working out and running - so what gyms are in the area? How close are theses areas to my work?  are they close to other major landmarks like close to lake michica or Wrigleyville?  



## Data to be incorporated

There are three primary data sources I will be using.  

The first is the active real estate data with is currently listed on Redfin.com.  From redfin I will be able to query all currnet active listings and a number of their attributes, such as price, bedrooms, neighborhood, HOA, etc.  I will load these listings into my python script.  in addtion to the standard data in the redfin data feed, i will be adding an number other variables such as mortgage payments and total monthly budget for living in this property.

The second will be the crime data for the city of Chicago, this is available through a number of sources such as data.cityofchicago.org.  i will agregate this data into the neigborhoods and develop a neighborhood score to append to the current realestate listing. 

The third and final data source will be the fourspare API for the general exploration of the negiborhoods. Using the potential housing available I will explore the different neighborhoods and ultimately choose the best houses to make offers on and in the final report I will outline the strengths and weaknesses of a number of the final options.


I am starting with downloading the bulk of the packages that will be used to run this project.  some will prob not get used in these innitial steps, but they have all been part of the course thus far and will likely get used over the next few exercises.  I have used the notes from the course as to what each one does.

In [5]:
from bs4 import BeautifulSoup # library for web scraping
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [6]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans


print('Libraries imported.')

Libraries imported.


In [7]:
import os
os.chdir('/Users/dfishe/Downloads')
data=pd.read_csv('redfin_2020-08-08-11-28-13.csv', header=0) #i downloaded the csv and uploaded it as a data frame

In [8]:
data.head()

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,MLS Listing,NaN,Condo/Co-op,50 E 16th St #1210,Chicago,IL,60616,249500,1,1.0,...,Active,NaN,NaN,http://www.redfin.com/IL/Chicago/50-E-16th-St-...,MRED,10811119,N,Y,41.860515,-87.625178
1,MLS Listing,NaN,Condo/Co-op,5618 W Lawrence Ave Unit 1W,Chicago,IL,60630,379900,4,2.5,...,Active,August-9-2020 11:00 AM,August-9-2020 01:00 PM,http://www.redfin.com/IL/Chicago/5618-W-Lawren...,MRED,10804570,N,Y,41.967951,-87.768377
2,MLS Listing,NaN,Condo/Co-op,901 S Plymouth Ct #1404,Chicago,IL,60605,242500,1,1.0,...,Active,NaN,NaN,http://www.redfin.com/IL/Chicago/901-S-Plymout...,MRED,10810681,N,Y,41.870153,-87.628066
3,MLS Listing,NaN,Condo/Co-op,1310 N Ritchie Ct Unit 21A,Chicago,IL,60610,489000,2,2.0,...,Active,NaN,NaN,http://www.redfin.com/IL/Chicago/1310-N-Ritchi...,MRED,10806296,N,Y,41.906487,-87.626850
4,MLS Listing,NaN,Condo/Co-op,3815 S Langley Ave #301,Chicago,IL,60653,200000,2,1.5,...,Active,NaN,NaN,http://www.redfin.com/IL/Chicago/3815-S-Langle...,MRED,10810443,N,Y,41.825265,-87.609451


In [10]:
address = 'Chicago'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.8755616 -87.6244212


In [17]:
search_query = 'Golds Gym'
radius = 500
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

Golds Gym .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=KARNGMJWOURD3LZYPRHGS3IEKZUZQF351EMRPWCYLI1BLKRJ&client_secret=JXEXLAJQH3JQ2PFEWFPF1GJ21OILUCMHAEL0Y5MTPQTWCJT1&ll=41.8755616,-87.6244212&v=20180604&query=Golds Gym&radius=500&limit=30'

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f2efb84d602790ae6fcf6e0'},
 'response': {'venues': [{'id': '50ee061ae4b0ebd26e9baff0',
    'name': 'The Buckingham Gym',
    'location': {'address': '59 E Van Buren St',
     'crossStreet': 'S Wabash Ave',
     'lat': 41.87688064575195,
     'lng': -87.6254653930664,
     'labeledLatLngs': [{'label': 'display',
       'lat': 41.87688064575195,
       'lng': -87.6254653930664}],
     'distance': 170,
     'postalCode': '60605',
     'cc': 'US',
     'city': 'Chicago',
     'state': 'IL',
     'country': 'United States',
     'formattedAddress': ['59 E Van Buren St (S Wabash Ave)',
      'Chicago, IL 60605',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d176941735',
      'name': 'Gym',
      'pluralName': 'Gyms',
      'shortName': 'Gym',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1596914571',
    'hasPerk': False},
   {'

In [19]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()
#below is a list of all of the thai venues

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,50ee061ae4b0ebd26e9baff0,The Buckingham Gym,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1596914571,False,59 E Van Buren St,S Wabash Ave,41.876881,-87.625465,"[{'label': 'display', 'lat': 41.87688064575195...",170,60605,US,Chicago,IL,United States,"[59 E Van Buren St (S Wabash Ave), Chicago, IL...",NaN
1,5239ceb50493e46a091a025d,Fit Results - Printers Row Gym,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1596914571,False,620 S Dearborn St,Harrison,41.873935,-87.629211,"[{'label': 'display', 'lat': 41.87393487095535...",436,60605,US,Chicago,IL,United States,"[620 S Dearborn St (Harrison), Chicago, IL 606...",73523364
2,4c6ec5f36af58cfa3a518d17,Astoria Tower Gym,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1596914571,False,NaN,NaN,41.870702,-87.627203,"[{'label': 'display', 'lat': 41.87070184798173...",588,60605,US,Chicago,IL,United States,"[Chicago, IL 60605, United States]",NaN
3,4ebc931261af9815e0c879d7,University Center Of Chicago Gym (2nd Floor),"[{'id': '4bf58dd8d48988d1b2941735', 'name': 'C...",v-1596914571,False,"525 S State St, Chicago, IL 60605",NaN,41.875160,-87.627098,"[{'label': 'display', 'lat': 41.87516021728515...",226,60605,US,Chicago,IL,United States,"[525 S State St, Chicago, IL 60605, Chicago, I...",NaN
4,5088b9f5e4b0f0fd18334f04,Metropolitan Tower Gym,"[{'id': '4bf58dd8d48988d1b2941735', 'name': 'C...",v-1596914571,False,NaN,NaN,41.878053,-87.624955,"[{'label': 'display', 'lat': 41.87805304432935...",280,60604,US,Chicago,IL,United States,"[Chicago, IL 60604, United States]",NaN
